In [27]:
import os
import glob
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision 
import pathlib

In [28]:
#checking for a device 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [29]:
device

device(type='cpu')

In [30]:
#transform

transformer = transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(), #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5, 0.5], #0-1 to [-1,1], formula (x-mean)/std
                        [0.5,0.5, 0.5])
])

In [31]:
#DataLoader
train_path = 'D:\hppl\seg_train/seg_train'
test_path = 'D:\hppl\seg_test/seg_test'

train_loader = DataLoader(
    torchvision.datasets.ImageFolder(train_path, transform=transformer),
    batch_size = 256, shuffle=True
)

train_loader = DataLoader(
    torchvision.datasets.ImageFolder(test_path, transform=transformer),
    batch_size = 256, shuffle=True
)

In [32]:
#categories
root = pathlib.Path(train_path)
classes = sorted([j.name for j in root.iterdir()])
classes

['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']

In [33]:
#CNN Network

class ConvNet(nn.Module):
    def __init__(self, num_classes=6):
        super(ConvNet, self).__init__()
        
        
        #Input image = (256(Batch size), 3(RGB), 150,150)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3,stride=1, padding=1)
        #Shape (256, 12,150,150)
        self.bn1 = nn.BatchNorm2d(num_features=12)
        #Shape (256, 12,150,150)
        self.relu1 = nn.ReLU()
        #Shape (256, 12,150,150)
        
        self.pool = nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape (256, 12,75,75)
        
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=20, kernel_size=3,stride=1, padding=1)
        #Shape (256, 20,75,75)
        self.relu2 = nn.ReLU()
        #Shape (256, 12,150,150)
        
        
        self.conv3 = nn.Conv2d(in_channels=20, out_channels=32, kernel_size=3,stride=1, padding=1)
        #Shape (256, 32,75,75)
        self.bn3 = nn.BatchNorm2d(num_features=32)
        #Shape (256, 32,75,75)
        self.relu3 = nn.ReLU()
        #Shape (256, 32,75,75)
        
        self.fc = nn.Linear(in_features = 32*75*75, out_features = num_classes)
        
    def forward(self,input):

        output = self.conv1(input)
        output = self.bn1(output)
        output = self.relu1(output)

        output = self.pool(output)

        output = self.conv2(output)
        output = self.relu2(output)

        output = self.conv3(output)
        output = self.bn3(output)
        output = self.relu3(output)

        #Above output will be in matrix form, with shape (256, 32,75,75)

        output = output.view(-1, 32*75*75)

        output = self.fc(output)

        return output
        
        
        

In [34]:
model = ConvNet(num_classes=6).to(device)

In [35]:
optimizer = Adam(model.parameters(), lr = 0.001, weight_decay=0.00001)
loss_function= nn.CrossEntropyLoss()

In [36]:
num_epochs = 10

In [37]:
#Calculating the size of training and testing images
train_count = len(glob.glob(train_path+'/**/*.jpg'))
test_count = len(glob.glob(test_path+'/**/*.jpg'))
train_count,test_count

(14034, 3000)

In [38]:
#Model training and saving best models



best_accuracy =0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss =0.0
    
    for i, (images,labels) in enumerate(train_loader):
        
        optimizer.zero_grad()
        outputs = model(images)
        
        loss = loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        train_loss+=loss.cpu().data*images.size(0)
        _,prediction = torch.max(outputs.data,1)
        
        train_accuracy+= int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
        
    
    
    #Evaluation on testing dataset
    model.eval()
    
    test_accuracy = 0.0
    
    for i, (images,labels) in enumerate(train_loader):
        
        outputs = model(images)
        _, prediction = torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
        
    test_accuracy = test_accuracy/test_count
    
    
    print(f'Epoch: {epoch} Train Loss: {int(train_loss)} Train Accuracy: {train_accuracy} Test Accuracy: {test_accuracy}')
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(), 'best_checkpoint.model')
        
        best_accuracy = test_accuracy
    
    
    
    


Epoch: 0 Train Loss: 7 Train Accuracy: 0.06398745902807468 Test Accuracy: 0.37866666666666665
Epoch: 1 Train Loss: 2 Train Accuracy: 0.10146786375944136 Test Accuracy: 0.22866666666666666
Epoch: 2 Train Loss: 1 Train Accuracy: 0.12355707567336469 Test Accuracy: 0.32266666666666666
Epoch: 3 Train Loss: 0 Train Accuracy: 0.1417984893829272 Test Accuracy: 0.542
Epoch: 4 Train Loss: 0 Train Accuracy: 0.16132250249394328 Test Accuracy: 0.5753333333333334
Epoch: 5 Train Loss: 0 Train Accuracy: 0.17799629471284026 Test Accuracy: 0.8233333333333334
Epoch: 6 Train Loss: 0 Train Accuracy: 0.19295995439646574 Test Accuracy: 0.931
Epoch: 7 Train Loss: 0 Train Accuracy: 0.1956676642439789 Test Accuracy: 0.923
Epoch: 8 Train Loss: 0 Train Accuracy: 0.2003705287159755 Test Accuracy: 0.9406666666666667
Epoch: 9 Train Loss: 0 Train Accuracy: 0.20485962662106313 Test Accuracy: 0.973


In [ ]:
class A:
    def __init__(self,first_name):
        self.first_name = first_name

In [68]:
class B(A):
    def __init__(self, first_name, last):
        super(B, self).__init__(first_name)
        self.last = last
        print(f'{first_name} {last}')
    
    

In [69]:
a = B('A', 'B')

A B
